# Task 2: Exploratory Data Analysis

## Objective
Analyze the data to understand patterns and factors influencing financial inclusion in Ethiopia.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['font.size'] = 10

# Paths
DATA_DIR = Path('../data/raw')
PROCESSED_DIR = Path('../data/processed')
REPORTS_DIR = Path('../reports/figures')
REPORTS_DIR.mkdir(exist_ok=True, parents=True)

print("Libraries loaded successfully")

## 1. Load Data

In [ ]:
# Load enriched dataset (or original if enrichment not done)
if (PROCESSED_DIR / 'ethiopia_fi_unified_data_enriched.csv').exists():
    df = pd.read_csv(PROCESSED_DIR / 'ethiopia_fi_unified_data_enriched.csv')
    print("Loaded enriched dataset")
else:
    df = pd.read_csv(DATA_DIR / 'ethiopia_fi_unified_data.csv')
    print("Loaded original dataset")

print(f"Dataset shape: {df.shape}")
print(f"Columns: {list(df.columns)}")

## 2. Dataset Overview

In [ ]:
# Summary by record_type
print("=== Record Type Distribution ===")
record_summary = df['record_type'].value_counts()
print(record_summary)
print(f"\nTotal records: {len(df)}")

# Visualize
fig, ax = plt.subplots(figsize=(10, 6))
record_summary.plot(kind='bar', ax=ax, color='steelblue')
ax.set_title('Distribution of Records by Type', fontsize=14, fontweight='bold')
ax.set_xlabel('Record Type', fontsize=12)
ax.set_ylabel('Count', fontsize=12)
ax.tick_params(axis='x', rotation=45)
plt.tight_layout()
plt.savefig(REPORTS_DIR / 'record_type_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Summary by pillar
if 'pillar' in df.columns:
    print("=== Pillar Distribution ===")
    pillar_summary = df[df['pillar'].notna()]['pillar'].value_counts()
    print(pillar_summary)
    
    fig, ax = plt.subplots(figsize=(10, 6))
    pillar_summary.plot(kind='bar', ax=ax, color='coral')
    ax.set_title('Distribution of Records by Pillar', fontsize=14, fontweight='bold')
    ax.set_xlabel('Pillar', fontsize=12)
    ax.set_ylabel('Count', fontsize=12)
    ax.tick_params(axis='x', rotation=45)
    plt.tight_layout()
    plt.savefig(REPORTS_DIR / 'pillar_distribution.png', dpi=300, bbox_inches='tight')
    plt.show()

In [ ]:
# Summary by source_type
if 'source_type' in df.columns:
    print("=== Source Type Distribution ===")
    source_summary = df['source_type'].value_counts()
    print(source_summary)
    
    fig, ax = plt.subplots(figsize=(12, 6))
    source_summary.plot(kind='bar', ax=ax, color='mediumseagreen')
    ax.set_title('Distribution of Records by Source Type', fontsize=14, fontweight='bold')
    ax.set_xlabel('Source Type', fontsize=12)
    ax.set_ylabel('Count', fontsize=12)
    ax.tick_params(axis='x', rotation=45)
    plt.tight_layout()
    plt.savefig(REPORTS_DIR / 'source_type_distribution.png', dpi=300, bbox_inches='tight')
    plt.show()

In [ ]:
# Confidence distribution
if 'confidence' in df.columns:
    print("=== Confidence Distribution ===")
    conf_summary = df['confidence'].value_counts()
    print(conf_summary)
    
    fig, ax = plt.subplots(figsize=(10, 6))
    conf_summary.plot(kind='bar', ax=ax, color='gold')
    ax.set_title('Distribution of Records by Confidence Level', fontsize=14, fontweight='bold')
    ax.set_xlabel('Confidence Level', fontsize=12)
    ax.set_ylabel('Count', fontsize=12)
    ax.tick_params(axis='x', rotation=45)
    plt.tight_layout()
    plt.savefig(REPORTS_DIR / 'confidence_distribution.png', dpi=300, bbox_inches='tight')
    plt.show()

## 3. Temporal Coverage Analysis

In [ ]:
# Identify date columns
date_cols = [col for col in df.columns if 'date' in col.lower()]
print(f"Date columns found: {date_cols}")

# Convert to datetime
for col in date_cols:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors='coerce')

# Create temporal coverage visualization
observations = df[df['record_type'] == 'observation'].copy()
if 'observation_date' in observations.columns:
    observations = observations[observations['observation_date'].notna()]
    
    # Get unique indicators and years
    if 'indicator_code' in observations.columns:
        indicators = observations['indicator_code'].unique()
        years = observations['observation_date'].dt.year.unique()
        
        # Create coverage matrix
        coverage = pd.DataFrame(index=sorted(indicators), columns=sorted(years))
        for ind in indicators:
            ind_data = observations[observations['indicator_code'] == ind]
            for year in years:
                coverage.loc[ind, year] = len(ind_data[ind_data['observation_date'].dt.year == year])
        
        coverage = coverage.fillna(0)
        
        # Visualize
        plt.figure(figsize=(14, max(8, len(indicators) * 0.5)))
        sns.heatmap(coverage.astype(int), annot=True, fmt='d', cmap='YlOrRd', cbar_kws={'label': 'Number of Observations'})
        plt.title('Temporal Coverage by Indicator', fontsize=14, fontweight='bold')
        plt.xlabel('Year', fontsize=12)
        plt.ylabel('Indicator Code', fontsize=12)
        plt.tight_layout()
        plt.savefig(REPORTS_DIR / 'temporal_coverage.png', dpi=300, bbox_inches='tight')
        plt.show()
        
        print(f"\nTemporal range: {observations['observation_date'].min()} to {observations['observation_date'].max()}")
        print(f"Indicators with data: {len(indicators)}")
        print(f"Years covered: {sorted(years)}")

## 4. Access Analysis - Account Ownership

In [ ]:
# Extract account ownership data
acc_obs = observations[
    (observations['indicator_code'].str.contains('ACC_OWNERSHIP', case=False, na=False)) |
    (observations['indicator'] == 'Account Ownership Rate')
].copy()

if len(acc_obs) > 0 and 'observation_date' in acc_obs.columns:
    acc_obs = acc_obs.sort_values('observation_date')
    
    # Plot trajectory
    fig, ax = plt.subplots(figsize=(14, 8))
    
    if 'value_numeric' in acc_obs.columns:
        ax.plot(acc_obs['observation_date'], acc_obs['value_numeric'], 
               marker='o', markersize=10, linewidth=2, color='steelblue', label='Account Ownership')
        ax.scatter(acc_obs['observation_date'], acc_obs['value_numeric'], 
                  s=200, zorder=5, color='darkblue')
        
        # Add value labels
        for idx, row in acc_obs.iterrows():
            ax.annotate(f"{row['value_numeric']:.1f}%", 
                       (row['observation_date'], row['value_numeric']),
                       textcoords="offset points", xytext=(0,15), ha='center', fontsize=10)
    
    ax.set_title('Ethiopia Account Ownership Trajectory (2011-2024)', 
                fontsize=16, fontweight='bold')
    ax.set_xlabel('Year', fontsize=12)
    ax.set_ylabel('Account Ownership Rate (%)', fontsize=12)
    ax.grid(True, alpha=0.3)
    ax.legend(fontsize=11)
    plt.tight_layout()
    plt.savefig(REPORTS_DIR / 'account_ownership_trajectory.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    # Calculate growth rates
    if len(acc_obs) > 1:
        acc_obs['year'] = acc_obs['observation_date'].dt.year
        acc_obs['growth_pp'] = acc_obs['value_numeric'].diff()
        acc_obs['growth_pct'] = acc_obs['value_numeric'].pct_change() * 100
        
        print("\n=== Account Ownership Growth Rates ===")
        for i in range(1, len(acc_obs)):
            prev_year = acc_obs.iloc[i-1]['year']
            curr_year = acc_obs.iloc[i]['year']
            growth_pp = acc_obs.iloc[i]['growth_pp']
            print(f"{prev_year}-{curr_year}: {growth_pp:+.1f} percentage points")
else:
    print("No account ownership data found")

### 4.1 Gender Gap Analysis

In [ ]:
# Check for gender-disaggregated data
gender_obs = observations[
    (observations['indicator_code'].str.contains('GENDER', case=False, na=False)) |
    (observations['indicator_code'].str.contains('MALE', case=False, na=False)) |
    (observations['indicator_code'].str.contains('FEMALE', case=False, na=False))
].copy()

if len(gender_obs) > 0:
    print(f"Found {len(gender_obs)} gender-disaggregated observations")
    print(gender_obs[['indicator_code', 'observation_date', 'value_numeric']].head(10))
    
    # Visualize if we have male/female data
    male_data = gender_obs[gender_obs['indicator_code'].str.contains('MALE', case=False, na=False)]
    female_data = gender_obs[gender_obs['indicator_code'].str.contains('FEMALE', case=False, na=False)]
    
    if len(male_data) > 0 and len(female_data) > 0:
        fig, ax = plt.subplots(figsize=(14, 8))
        ax.plot(male_data['observation_date'], male_data['value_numeric'], 
               marker='o', label='Male', linewidth=2, color='steelblue')
        ax.plot(female_data['observation_date'], female_data['value_numeric'], 
               marker='s', label='Female', linewidth=2, color='coral')
        ax.set_title('Account Ownership by Gender', fontsize=16, fontweight='bold')
        ax.set_xlabel('Year', fontsize=12)
        ax.set_ylabel('Account Ownership Rate (%)', fontsize=12)
        ax.legend(fontsize=11)
        ax.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.savefig(REPORTS_DIR / 'account_ownership_gender.png', dpi=300, bbox_inches='tight')
        plt.show()
else:
    print("No gender-disaggregated data found in current dataset")

### 4.2 Analyzing the 2021-2024 Slowdown

In [ ]:
# Analyze the slowdown: Account ownership grew only +3pp from 2021-2024
# despite massive mobile money expansion (65M+ accounts)

print("=== Analysis of 2021-2024 Slowdown ===")
print("\nKey Question: Why did account ownership grow only +3pp despite 65M+ mobile money accounts?")

# Get mobile money account data
mm_obs = observations[
    (observations['indicator_code'].str.contains('MM_ACCOUNT', case=False, na=False)) |
    (observations['indicator'].str.contains('mobile money', case=False, na=False))
].copy()

if len(mm_obs) > 0:
    print(f"\nMobile Money Account Observations: {len(mm_obs)}")
    print(mm_obs[['indicator_code', 'observation_date', 'value_numeric']].sort_values('observation_date'))
    
    # Compare account ownership vs mobile money accounts
    if len(acc_obs) > 0:
        fig, ax1 = plt.subplots(figsize=(14, 8))
        
        ax1.plot(acc_obs['observation_date'], acc_obs['value_numeric'], 
                marker='o', label='Account Ownership (%)', linewidth=2, color='steelblue')
        ax1.set_xlabel('Year', fontsize=12)
        ax1.set_ylabel('Account Ownership (%)', fontsize=12, color='steelblue')
        ax1.tick_params(axis='y', labelcolor='steelblue')
        ax1.grid(True, alpha=0.3)
        
        if len(mm_obs) > 0:
            ax2 = ax1.twinx()
            ax2.plot(mm_obs['observation_date'], mm_obs['value_numeric'], 
                    marker='s', label='Mobile Money Accounts (%)', linewidth=2, color='coral')
            ax2.set_ylabel('Mobile Money Accounts (%)', fontsize=12, color='coral')
            ax2.tick_params(axis='y', labelcolor='coral')
        
        ax1.set_title('Account Ownership vs Mobile Money Accounts', fontsize=16, fontweight='bold')
        ax1.legend(loc='upper left', fontsize=11)
        if len(mm_obs) > 0:
            ax2.legend(loc='upper right', fontsize=11)
        plt.tight_layout()
        plt.savefig(REPORTS_DIR / 'account_vs_mm_comparison.png', dpi=300, bbox_inches='tight')
        plt.show()

print("\nPotential Explanations:")
print("1. Registered vs. Active accounts gap")
print("2. Multiple accounts per person")
print("3. Low usage/activation rates")
print("4. Bank account ownership may have declined")
print("5. Survey methodology differences")

## 5. Usage Analysis - Digital Payments

In [ ]:
# Extract digital payment usage data
usage_obs = observations[
    (observations['indicator_code'].str.contains('USG_DIGITAL', case=False, na=False)) |
    (observations['indicator_code'].str.contains('DIGITAL_PAYMENT', case=False, na=False)) |
    (observations['pillar'] == 'usage')
].copy()

if len(usage_obs) > 0 and 'observation_date' in usage_obs.columns:
    usage_obs = usage_obs.sort_values('observation_date')
    
    # Plot trajectory
    fig, ax = plt.subplots(figsize=(14, 8))
    
    if 'value_numeric' in usage_obs.columns:
        ax.plot(usage_obs['observation_date'], usage_obs['value_numeric'], 
               marker='o', markersize=10, linewidth=2, color='mediumseagreen', label='Digital Payment Usage')
        ax.scatter(usage_obs['observation_date'], usage_obs['value_numeric'], 
                  s=200, zorder=5, color='darkgreen')
        
        # Add value labels
        for idx, row in usage_obs.iterrows():
            ax.annotate(f"{row['value_numeric']:.1f}%", 
                       (row['observation_date'], row['value_numeric']),
                       textcoords="offset points", xytext=(0,15), ha='center', fontsize=10)
    
    ax.set_title('Ethiopia Digital Payment Usage Trajectory', 
                fontsize=16, fontweight='bold')
    ax.set_xlabel('Year', fontsize=12)
    ax.set_ylabel('Digital Payment Usage Rate (%)', fontsize=12)
    ax.grid(True, alpha=0.3)
    ax.legend(fontsize=11)
    plt.tight_layout()
    plt.savefig(REPORTS_DIR / 'digital_payment_usage.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print(f"\nDigital Payment Usage Observations: {len(usage_obs)}")
    print(usage_obs[['indicator_code', 'observation_date', 'value_numeric']])
else:
    print("No digital payment usage data found")

### 5.1 Mobile Money Account Penetration

In [ ]:
# Mobile money account penetration trend
if len(mm_obs) > 0 and 'observation_date' in mm_obs.columns:
    mm_obs = mm_obs.sort_values('observation_date')
    
    fig, ax = plt.subplots(figsize=(14, 8))
    ax.plot(mm_obs['observation_date'], mm_obs['value_numeric'], 
           marker='o', markersize=10, linewidth=2, color='purple', label='Mobile Money Account Ownership')
    ax.scatter(mm_obs['observation_date'], mm_obs['value_numeric'], 
              s=200, zorder=5, color='darkviolet')
    
    # Add value labels
    for idx, row in mm_obs.iterrows():
        ax.annotate(f"{row['value_numeric']:.1f}%", 
                   (row['observation_date'], row['value_numeric']),
                   textcoords="offset points", xytext=(0,15), ha='center', fontsize=10)
    
    ax.set_title('Mobile Money Account Ownership Trend (2014-2024)', 
                fontsize=16, fontweight='bold')
    ax.set_xlabel('Year', fontsize=12)
    ax.set_ylabel('Mobile Money Account Ownership (%)', fontsize=12)
    ax.grid(True, alpha=0.3)
    ax.legend(fontsize=11)
    plt.tight_layout()
    plt.savefig(REPORTS_DIR / 'mm_account_trend.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    # Calculate growth
    if len(mm_obs) > 1:
        mm_obs['year'] = mm_obs['observation_date'].dt.year
        mm_obs['growth_pp'] = mm_obs['value_numeric'].diff()
        print("\n=== Mobile Money Account Growth ===")
        for i in range(1, len(mm_obs)):
            prev_year = mm_obs.iloc[i-1]['year']
            curr_year = mm_obs.iloc[i]['year']
            growth_pp = mm_obs.iloc[i]['growth_pp']
            print(f"{prev_year}-{curr_year}: {growth_pp:+.1f} percentage points")

## 6. Infrastructure and Enablers Analysis

In [ ]:
# Extract infrastructure data
infra_obs = observations[
    (observations['indicator_code'].str.contains('INF_', case=False, na=False)) |
    (observations['indicator'].str.contains('coverage', case=False, na=False)) |
    (observations['indicator'].str.contains('penetration', case=False, na=False)) |
    (observations['indicator'].str.contains('ATM', case=False, na=False)) |
    (observations['indicator'].str.contains('agent', case=False, na=False))
].copy()

if len(infra_obs) > 0:
    print(f"Infrastructure observations: {len(infra_obs)}")
    print("\nInfrastructure indicators:")
    print(infra_obs['indicator_code'].value_counts())
    
    # Visualize infrastructure trends
    if 'observation_date' in infra_obs.columns:
        infra_obs = infra_obs.sort_values('observation_date')
        
        # Group by indicator
        for indicator in infra_obs['indicator_code'].unique():
            ind_data = infra_obs[infra_obs['indicator_code'] == indicator]
            if len(ind_data) > 1:
                fig, ax = plt.subplots(figsize=(12, 6))
                ax.plot(ind_data['observation_date'], ind_data['value_numeric'], 
                       marker='o', linewidth=2, label=indicator)
                ax.set_title(f'{indicator} Trend', fontsize=14, fontweight='bold')
                ax.set_xlabel('Year', fontsize=12)
                ax.set_ylabel('Value', fontsize=12)
                ax.grid(True, alpha=0.3)
                plt.tight_layout()
                plt.savefig(REPORTS_DIR / f'infra_{indicator}.png', dpi=300, bbox_inches='tight')
                plt.show()
else:
    print("Limited infrastructure data in current dataset")

## 7. Event Timeline and Visual Analysis

In [ ]:
# Extract events
events = df[df['record_type'] == 'event'].copy()
if 'event_date' in events.columns:
    events['event_date'] = pd.to_datetime(events['event_date'], errors='coerce')
elif 'observation_date' in events.columns:
    events['event_date'] = pd.to_datetime(events['observation_date'], errors='coerce')
    events.rename(columns={'observation_date': 'event_date'}, inplace=True)

if len(events) > 0 and 'event_date' in events.columns:
    events = events[events['event_date'].notna()].sort_values('event_date')
    
    print(f"Total events: {len(events)}")
    print("\nEvents timeline:")
    print(events[['event_date', 'category', 'event_name']].to_string())
    
    # Create timeline visualization
    fig, ax = plt.subplots(figsize=(16, max(8, len(events) * 0.8)))
    
    # Color by category
    category_colors = {
        'policy': 'steelblue',
        'product_launch': 'coral',
        'infrastructure': 'mediumseagreen',
        'market_entry': 'purple',
        'milestone': 'gold'
    }
    
    y_pos = 0
    for idx, event in events.iterrows():
        category = event.get('category', 'other')
        color = category_colors.get(category, 'gray')
        
        ax.scatter(event['event_date'], y_pos, s=300, color=color, zorder=5)
        ax.annotate(f"{event['event_date'].strftime('%Y-%m')}: {event.get('event_name', 'Event')}", 
                   (event['event_date'], y_pos),
                   textcoords="offset points", xytext=(10, 0), ha='left', fontsize=10)
        y_pos += 1
    
    ax.set_yticks([])
    ax.set_xlabel('Date', fontsize=12)
    ax.set_title('Event Timeline', fontsize=16, fontweight='bold')
    ax.grid(True, alpha=0.3, axis='x')
    
    # Add legend
    from matplotlib.patches import Patch
    legend_elements = [Patch(facecolor=color, label=cat) 
                      for cat, color in category_colors.items() 
                      if cat in events['category'].values]
    ax.legend(handles=legend_elements, loc='upper right', fontsize=10)
    
    plt.tight_layout()
    plt.savefig(REPORTS_DIR / 'event_timeline.png', dpi=300, bbox_inches='tight')
    plt.show()
else:
    print("No events found or event dates missing")

### 7.1 Events Overlaid on Indicator Trends

In [ ]:
# Overlay events on account ownership trend
if len(acc_obs) > 0 and len(events) > 0 and 'event_date' in events.columns:
    fig, ax = plt.subplots(figsize=(16, 10))
    
    # Plot account ownership
    ax.plot(acc_obs['observation_date'], acc_obs['value_numeric'], 
           marker='o', markersize=10, linewidth=3, color='steelblue', label='Account Ownership', zorder=3)
    
    # Add events as vertical lines
    for idx, event in events.iterrows():
        event_date = event['event_date']
        event_name = event.get('event_name', 'Event')
        category = event.get('category', 'other')
        color = category_colors.get(category, 'gray')
        
        ax.axvline(event_date, color=color, linestyle='--', alpha=0.6, linewidth=2)
        ax.annotate(event_name, 
                   (event_date, ax.get_ylim()[1] * 0.95),
                   rotation=90, ha='right', fontsize=9, color=color, fontweight='bold')
    
    ax.set_title('Account Ownership Trend with Key Events', fontsize=16, fontweight='bold')
    ax.set_xlabel('Year', fontsize=12)
    ax.set_ylabel('Account Ownership Rate (%)', fontsize=12)
    ax.legend(fontsize=11)
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig(REPORTS_DIR / 'account_ownership_with_events.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    # Analyze specific events
    print("\n=== Key Event Analysis ===")
    print("\nTelebirr Launch (May 2021):")
    print("  - Mobile money accounts: 4.7% (2021) → 9.45% (2024)")
    print("  - Account ownership: 46% (2021) → 49% (2024)")
    print("  - Impact: Significant growth in mobile money, but limited overall account ownership growth")
    
    if len(events[events['event_name'].str.contains('M-Pesa', case=False, na=False)]) > 0:
        print("\nM-Pesa Entry (Aug 2023):")
        print("  - Too recent to fully assess impact")
        print("  - Expected to increase competition and usage")

## 8. Correlation Analysis

In [ ]:
# Create pivot table of observations by indicator and year
if 'indicator_code' in observations.columns and 'observation_date' in observations.columns:
    observations['year'] = observations['observation_date'].dt.year
    
    # Pivot to get indicators as columns and years as rows
    pivot_data = observations.pivot_table(
        index='year',
        columns='indicator_code',
        values='value_numeric',
        aggfunc='mean'
    )
    
    if len(pivot_data) > 1:
        # Calculate correlation matrix
        corr_matrix = pivot_data.corr()
        
        # Visualize correlation matrix
        plt.figure(figsize=(max(12, len(corr_matrix) * 0.8), max(10, len(corr_matrix) * 0.8)))
        sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm', center=0,
                   square=True, linewidths=1, cbar_kws={'label': 'Correlation Coefficient'})
        plt.title('Correlation Matrix: Financial Inclusion Indicators', fontsize=16, fontweight='bold')
        plt.tight_layout()
        plt.savefig(REPORTS_DIR / 'correlation_matrix.png', dpi=300, bbox_inches='tight')
        plt.show()
        
        # Identify strong correlations with Access
        if 'ACC_OWNERSHIP' in corr_matrix.columns:
            acc_corr = corr_matrix['ACC_OWNERSHIP'].sort_values(ascending=False)
            print("\n=== Correlations with Account Ownership ===")
            print(acc_corr[acc_corr.index != 'ACC_OWNERSHIP'].head(10))
        
        # Identify strong correlations with Usage
        usage_cols = [col for col in corr_matrix.columns if 'USG' in col or 'DIGITAL' in col]
        if usage_cols:
            print("\n=== Correlations with Digital Payment Usage ===")
            for col in usage_cols:
                usage_corr = corr_matrix[col].sort_values(ascending=False)
                print(f"\n{col}:")
                print(usage_corr[usage_corr.index != col].head(5))
    else:
        print("Insufficient data for correlation analysis (need multiple years)")
else:
    print("Missing required columns for correlation analysis")

## 9. Impact Links Analysis

In [ ]:
# Analyze impact links
impact_links = df[df['record_type'] == 'impact_link'].copy()

if len(impact_links) > 0:
    print(f"Total impact links: {len(impact_links)}")
    
    # Summary by pillar
    if 'pillar' in impact_links.columns:
        print("\n=== Impact Links by Pillar ===")
        print(impact_links['pillar'].value_counts())
    
    # Summary by impact direction
    if 'impact_direction' in impact_links.columns:
        print("\n=== Impact Links by Direction ===")
        print(impact_links['impact_direction'].value_counts())
    
    # Create impact matrix
    if 'parent_id' in impact_links.columns and 'related_indicator' in impact_links.columns:
        impact_matrix = impact_links.pivot_table(
            index='parent_id',
            columns='related_indicator',
            values='impact_magnitude',
            aggfunc='mean'
        )
        
        if len(impact_matrix) > 0:
            plt.figure(figsize=(max(14, len(impact_matrix.columns) * 1.5), 
                               max(8, len(impact_matrix) * 0.8)))
            sns.heatmap(impact_matrix, annot=True, fmt='.2f', cmap='RdYlGn', center=0,
                       square=False, linewidths=1, cbar_kws={'label': 'Impact Magnitude'})
            plt.title('Event-Indicator Impact Matrix', fontsize=16, fontweight='bold')
            plt.xlabel('Indicator', fontsize=12)
            plt.ylabel('Event (parent_id)', fontsize=12)
            plt.tight_layout()
            plt.savefig(REPORTS_DIR / 'impact_matrix.png', dpi=300, bbox_inches='tight')
            plt.show()
            
            print("\n=== Impact Matrix Summary ===")
            print(impact_matrix)
else:
    print("No impact links found")

## 10. Key Insights Summary

In [ ]:
# Compile key insights
insights = []

print("=" * 80)
print("KEY INSIGHTS FROM EXPLORATORY DATA ANALYSIS")
print("=" * 80)

# Insight 1: Account Ownership Trajectory
if len(acc_obs) > 0:
    insights.append({
        'insight': 'Account Ownership Growth Pattern',
        'evidence': f"Account ownership grew from {acc_obs['value_numeric'].min():.1f}% to {acc_obs['value_numeric'].max():.1f}% over the period",
        'implication': 'Steady growth but with deceleration in recent period'
    })
    print(f"\n1. {insights[-1]['insight']}")
    print(f"   Evidence: {insights[-1]['evidence']}")
    print(f"   Implication: {insights[-1]['implication']}")

# Insight 2: 2021-2024 Slowdown
insights.append({
    'insight': '2021-2024 Slowdown Despite Mobile Money Expansion',
    'evidence': 'Account ownership grew only +3pp (46% to 49%) despite 65M+ mobile money accounts',
    'implication': 'Registered accounts ≠ active usage; need to focus on activation and usage'
})
print(f"\n2. {insights[-1]['insight']}")
print(f"   Evidence: {insights[-1]['evidence']}")
print(f"   Implication: {insights[-1]['implication']}")

# Insight 3: Mobile Money Growth
if len(mm_obs) > 0:
    insights.append({
        'insight': 'Mobile Money Account Rapid Growth',
        'evidence': f"Mobile money accounts grew from {mm_obs['value_numeric'].min():.1f}% to {mm_obs['value_numeric'].max():.1f}%",
        'implication': 'Mobile money is the primary driver of new account openings'
    })
    print(f"\n3. {insights[-1]['insight']}")
    print(f"   Evidence: {insights[-1]['evidence']}")
    print(f"   Implication: {insights[-1]['implication']}")

# Insight 4: Event Impact
if len(events) > 0:
    insights.append({
        'insight': 'Key Events Shaping Financial Inclusion',
        'evidence': f"Identified {len(events)} major events including Telebirr launch, M-Pesa entry",
        'implication': 'Product launches and market competition are key drivers'
    })
    print(f"\n4. {insights[-1]['insight']}")
    print(f"   Evidence: {insights[-1]['evidence']}")
    print(f"   Implication: {insights[-1]['implication']}")

# Insight 5: Data Gaps
insights.append({
    'insight': 'Data Limitations',
    'evidence': 'Limited infrastructure data, sparse time series, few disaggregated observations',
    'implication': 'Forecasts will have higher uncertainty; need to leverage comparable country evidence'
})
print(f"\n5. {insights[-1]['insight']}")
print(f"   Evidence: {insights[-1]['evidence']}")
print(f"   Implication: {insights[-1]['implication']}")

print("\n" + "=" * 80)

# Save insights to file
insights_df = pd.DataFrame(insights)
insights_df.to_csv(REPORTS_DIR / 'key_insights.csv', index=False)
print(f"\nInsights saved to {REPORTS_DIR / 'key_insights.csv'}")

## 11. Data Quality Assessment

In [ ]:
# Comprehensive data quality assessment
print("=" * 80)
print("DATA QUALITY ASSESSMENT")
print("=" * 80)

# Missing values
print("\n1. Missing Values:")
missing = df.isnull().sum()
missing_pct = (missing / len(df) * 100).round(2)
missing_df = pd.DataFrame({
    'Missing Count': missing,
    'Missing %': missing_pct
})
print(missing_df[missing_df['Missing Count'] > 0].sort_values('Missing Count', ascending=False).head(10))

# Temporal coverage gaps
print("\n2. Temporal Coverage:")
if 'observation_date' in observations.columns:
    obs_years = sorted(observations['observation_date'].dt.year.unique())
    print(f"   Years with observations: {obs_years}")
    print(f"   Total years covered: {len(obs_years)}")
    print(f"   Gaps: {[y for y in range(obs_years[0], obs_years[-1]+1) if y not in obs_years]}")

# Indicator coverage
print("\n3. Indicator Coverage:")
if 'indicator_code' in observations.columns:
    indicators = observations['indicator_code'].unique()
    print(f"   Unique indicators: {len(indicators)}")
    indicator_counts = observations['indicator_code'].value_counts()
    print(f"   Indicators with < 3 observations: {len(indicator_counts[indicator_counts < 3])}")

# Confidence levels
print("\n4. Data Confidence:")
if 'confidence' in df.columns:
    conf_dist = df['confidence'].value_counts()
    print(conf_dist)
    high_conf_pct = (conf_dist.get('high', 0) / len(df) * 100).round(1)
    print(f"   High confidence records: {high_conf_pct}%")

# Source diversity
print("\n5. Source Diversity:")
if 'source_name' in df.columns:
    sources = df['source_name'].value_counts()
    print(f"   Unique sources: {len(sources)}")
    print(f"   Top 5 sources:")
    print(sources.head(5))

print("\n" + "=" * 80)
print("DATA QUALITY LIMITATIONS:")
print("=" * 80)
print("1. Sparse time series: Only 5 Findex survey points over 13 years")
print("2. Limited infrastructure data: Need more enabler indicators")
print("3. Few disaggregated observations: Limited gender/urban-rural data")
print("4. Event impact estimates: Many based on comparable country evidence")
print("5. Recent events: M-Pesa entry too recent to fully assess impact")
print("=" * 80)